In [1]:
from typing import List, Optional

import fire

from llama import Dialog, Llama
import torch.distributed as dist
import torch
import os
from data_casual import output_list_train, input_list_train

In [2]:

from llama.model import ModelArgs, Transformer
from llama.tokenizer import ChatFormat, Dialog, Message, Tokenizer

In [3]:
import json
import os
import sys
import time
from pathlib import Path
from typing import List, Optional, Tuple, TypedDict

import torch
import torch.nn.functional as F
from fairscale.nn.model_parallel.initialize import (
    get_model_parallel_rank,
    initialize_model_parallel,
    model_parallel_is_initialized,
)

In [4]:
generator = Llama.build("./", "./tokenizer.model",max_seq_len= 2500,max_batch_size= 4, activation=True, activation_layer=28)

> initializing model parallel with size 1
> initializing ddp with size 1
> initializing pipeline with size 1


c:\Users\Ribbe\Coding\VSC\FU_Berlin\Actual_Work\llama3\llama\generation.py:98: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(ckpt_path, map_location=

Loaded in 231.73 seconds


In [5]:
ckpt_dir = "./"
tokenizer_path = "./tokenizer.model"
temperature = 0.6
top_p = 0.9
max_seq_len = 1024
max_batch_size = 4
max_gen_len= generator.model.params.max_seq_len - 1


In [6]:
def sample_top_p(probs, p):
    """
    Perform top-p (nucleus) sampling on a probability distribution.

    Args:
        probs (torch.Tensor): Probability distribution tensor.
        p (float): Probability threshold for top-p sampling.

    Returns:
        torch.Tensor: Sampled token indices.

    Note:
        Top-p sampling selects the smallest set of tokens whose cumulative probability mass
        exceeds the threshold p. The distribution is renormalized based on the selected tokens.
    """
    probs_sort, probs_idx = torch.sort(probs, dim=-1, descending=True)
    probs_sum = torch.cumsum(probs_sort, dim=-1)
    mask = probs_sum - probs_sort > p
    probs_sort[mask] = 0.0
    probs_sort.div_(probs_sort.sum(dim=-1, keepdim=True))
    next_token = torch.multinomial(probs_sort, num_samples=1)
    next_token = torch.gather(probs_idx, -1, next_token)
    return next_token

In [7]:
prompt_tokens = [generator.formatter.encode_dialog_prompt(input_list_train[1142])]

In [8]:
n = len(input_list_train)
n

205734

In [9]:
neg_activation_vector_dic = {}
pos_activation_vector_dic = {}
loss = 0
for i in range(n//50):
    value, activation_vec_list = generator.chat_completion([input_list_train[i]],
                                       max_gen_len=max_gen_len,
                                       top_p=top_p,
                                       temperature=temperature)
    value_numerical = "1" if value[0]["generation"]["content"] == "T" else 0
    if value_numerical == output_list_train[i]:
        pos_activation_vector_dic[i] = activation_vec_list[0] 
    else :
        neg_activation_vector_dic[i] = activation_vec_list[0]
        loss += 1/len(input_list_train)


In [10]:
pos_vec_sum = torch.zeros([len(neg_activation_vector_dic[1])])
for ind, item in pos_activation_vector_dic.items():
    pos_vec_sum += 1/len(pos_activation_vector_dic)*item


neg_vec_sum = torch.zeros([len(neg_activation_vector_dic[1])])
for ind, item in neg_activation_vector_dic.items():
    neg_vec_sum += 1/len(neg_activation_vector_dic)*item

steering_vector = pos_vec_sum-neg_vec_sum


In [11]:
print(steering_vector)

tensor([-0.2949,  0.0142, -0.0035,  ..., -0.1094,  0.0864,  0.1484])


In [12]:
torch.save(steering_vector, "./steering_vector_layer28.pt")